### PLAN
- GIL
- threading
- multiprocessing
- async

### Main problem: 
string normalization work takes a lot of time!

In [130]:
import re
import pymorphy2
import pandas as pd
morph = pymorphy2.MorphAnalyzer()
df = pd.read_excel('saratovsk_v1.xls')

def norm(s):
    morph = pymorphy2.MorphAnalyzer()
    s = re.sub(r'[\d\$\%\#\@\№\^&\*]', '', s)
    
    return ' '.join([morph.parse(i)[0].normal_form for i in s.split(' ')])
# df['fullName_norm'] = df['fullName'].apply(norm) ### HERE!!!!

In [12]:
df.head()

,inn,fullName,opfType,foundersType,allOkveds,regionName,egrulStatus,okveds_num
0,6.451901e+09,КООПЕРАТИВ №2 ПО СТРОИТЕЛЬСТВУ ЭКСПЛУАТАЦИИ КО...,Кооперативы и товарищества,Только физлица,52.21.24 Деятельность стоянок для транспортных...,Саратовская область,Действует,1.0
1,6.439071e+09,"САДОВОДЧЕСКОЕ ТОВАРИЩЕСТВО ""ЗОЛОТЫЕ ПЕСКИ""",Кооперативы и товарищества,Только физлица,01.12 Овощеводство; декоративное садоводство и...,Саратовская область,Ликвидирована,4.0
2,6.454030e+09,САРАТОВСКАЯ ОБЩЕСТВЕННАЯ ОРГАНИЗАЦИЯ ЦЕНТР ТВО...,Общественные организации,NaN,NaN,Саратовская область,Ликвидирована,NaN
3,6.450061e+09,САРАТОВСКОЕ РЕГИОНАЛЬНОЕ ОБЩЕСТВЕННОЕ ДВИЖЕНИЕ...,Общественные организации,NaN,NaN,Саратовская область,Ликвидирована,NaN
4,6.454093e+09,"БЛАГОТВОРИТЕЛЬНЫЙ ФОНД ""БОГАТСТВО ОТЧИЗНЫ""",Фонды,Только юрлица,"64.99 Предоставление прочих финансовых услуг, ...",Саратовская область,Ликвидирована,1.0


In [77]:
from multiprocessing import cpu_count, Process, Pool, Queue, current_process

In [2]:
cpu_count()

6

## Process

In [32]:
def print_func(continent='Asia'):
    print('The name of continent is : ', continent)

In [62]:
p1 = Process(target=print_func, args=('Сережа',))
#p1 = Process(print_func('Сережа'))

In [63]:
p1.start()

The name of continent is :  Сережа


In [64]:
p1.join()

In [44]:
for name in  ['America', 'Europe', 'Africa']:
    procs = []
    proc = Process(target=print_func, args=(name,))  # instantiating without any argument
    procs.append(proc)
    proc.start()

The name of continent is :  America
The name of continent is :  Europe
The name of continent is :  Africa


In [54]:
for i in procs:
    i.join()

## Queue

In [69]:
colors = ['red', 'green', 'blue', 'black']
cnt = 1
# instantiating a queue object
queue = Queue()

In [70]:
for color in colors:
    print('item no: ', cnt, ' ', color)
    queue.put(color)
    cnt += 1

item no:  1   red
item no:  2   green
item no:  3   blue
item no:  4   black


In [71]:
queue.qsize()

4

In [72]:
queue.empty()

False

In [73]:
cnt = 0
while not queue.empty():
    print('item no: ', cnt, ' ', queue.get())
    cnt += 1

item no:  0   red
item no:  1   green
item no:  2   blue
item no:  3   black


In [116]:
def print_func(tasks_to_accomplish, tasks_that_are_done):
    try:
        task = tasks_to_accomplish.get_nowait()
    
        print('The name of continent is : ', task)
    except queue.Empty:
        print("РАБОТЫ БОЛЬШЕ НЕТ") 
    else:    
        tasks_that_are_done.put(task + ' is done by ' + current_process().name)
    
tasks_to_accomplish = Queue()
tasks_that_are_done = Queue()

names = ['America', 'Europe', 'Africa', 'Name1', 'Name2','Name3','Name4','Name5','Name6','Name7','Name8','Name9','Name10']

number_of_task = len(names)
number_of_processes = 4

processes = []

In [117]:
for i in names:
    tasks_to_accomplish.put(i)

In [118]:
while not tasks_to_accomplish.empty():
    for w in range(number_of_processes):
        p = Process(target=print_func, args=(tasks_to_accomplish, tasks_that_are_done))
        processes.append(p)
        p.start()

The name of continent is :  America
The name of continent is :  Europe
The name of continent is :  Africa
The name of continent is :  Name1
The name of continent is :  Name2
The name of continent is :  Name3
The name of continent is :  Name4
The name of continent is :  Name5
The name of continent is :  Name6
The name of continent is :  Name7
The name of continent is :  Name8
The name of continent is :  Name9
The name of continent is :  Name10
РАБОТЫ БОЛЬШЕ НЕТ
РАБОТЫ БОЛЬШЕ НЕТ
РАБОТЫ БОЛЬШЕ НЕТ


In [119]:
for p in processes:
    p.join()

In [120]:
while not tasks_that_are_done.empty():
    print(tasks_that_are_done.get())

America is done by Process-136
Europe is done by Process-137
Africa is done by Process-138
Name1 is done by Process-139
Name2 is done by Process-140
Name3 is done by Process-141
Name4 is done by Process-142
Name5 is done by Process-143
Name6 is done by Process-144
Name7 is done by Process-145
Name8 is done by Process-146
Name9 is done by Process-147
Name10 is done by Process-148


### ИТОГ:

сложно и опасно 

### POOL

легко и безопасно

In [121]:
import time 

In [132]:
## baseline
start = time.time()
df2 = df.iloc[:1000,]
df2['fullName_norm'] = df2['fullName'].apply(norm)
print('execution time: ', time.time()-start)

execution time:  37.22282600402832


/mnt/c/Users/sergey.chuvakin/Desktop/env2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [134]:
start = time.time()
p = Pool(cpu_count()-1)
df2['fullName_norm'] = p.map(norm, df2['fullName'])
print('execution time: ', time.time()-start)
p.close()
p.join()


execution time:  13.593991756439209


/mnt/c/Users/sergey.chuvakin/Desktop/env2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [137]:
with Pool(cpu_count()-1) as pool:
    start = time.time()
    df2['fullName_norm'] = pool.map(norm, df2['fullName'])
    print('execution time: ', time.time()-start)

execution time:  14.549742937088013


/mnt/c/Users/sergey.chuvakin/Desktop/env2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### ASYNC
- корутины
- цикл событий 
- оператор await 

In [138]:
import asyncio

In [145]:
async def foo(name):
    await asyncio.sleep(1) 
    print('привет', name)

In [146]:
# asyncio.run(foo) # не работает в jupyter
await foo('Сережа')

привет Сережа


In [149]:
# nested 
async def boo(name='Сережа'): 
    print('привет', name)
async def main():
    await boo()
    
await main()

привет Сережа


In [150]:
# tasks 
async def boo(name='Сережа'): 
    print('привет', name)
async def foo(name='Сережа'):
    print('пока', name)
async def main():
    task1 = asyncio.create_task(boo())
    task2 = asyncio.create_task(foo())
    tasks = [task1, task2]
    await asyncio.gather(*tasks) # если функция должна что то возвращать, то необходимо добавить return
await main()

привет Сережа
пока Сережа


Problem: 
- читаем https://docs.aiohttp.org/en/stable/ 
- пишем ассинхронную функцию для запроса сюда http://myip.ru/index_small.php 
- сравните скорость 100 запросов, без ассинзхронности и с нею 

Problem: 
- читаем и пробуем у себя https://geekbrains.ru/posts/python_threading_part1